In [2]:
!pip3 install requests
!pip3 install bs4

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=4477ed5cc65d5339226798e4cf967348ee6289618e1e73835ef86cf10e45ea04
  Stored in directory: /home/gumeta/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [48]:
import requests
from bs4 import BeautifulSoup

In [49]:
# URL do site que será feito o scraping
url = 'https://www.kabum.com.br/computadores/monitores/monitor-gamer'

# Essa linha é necessária para que o site não bloqueie a requisição por ser um bot
headers = {'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"}

In [50]:
# Função para coletar o link da proxima pagina
def next_page(soup):
    # Ir na ultima pagina com o botao proximo desabilitado
    if not soup.find( 'li', {'class' : 'next disabled'} ):
        next = soup.find('link', {'rel': 'next'})
        return next['href'].replace('?', '/computadores/monitores?')
    else:
        return None

In [51]:

while True:
    
    # Faz a requisição do site e armazena o conteúdo na variável site
    site = requests.get(url, headers = headers)

    # Transforma o conteúdo do site (html) em um objeto BeautifulSoup
    soup = BeautifulSoup(site.content, 'html.parser')

    # Recebe o link da proxima pagina
    url = next_page(soup)
    print(url)
    if url == None:
        break

https://www.kabum.com.br/computadores/monitores?page_number=2
https://www.kabum.com.br/computadores/monitores?page_number=3
https://www.kabum.com.br/computadores/monitores?page_number=4
https://www.kabum.com.br/computadores/monitores?page_number=5
https://www.kabum.com.br/computadores/monitores?page_number=6
https://www.kabum.com.br/computadores/monitores?page_number=7
https://www.kabum.com.br/computadores/monitores?page_number=8
https://www.kabum.com.br/computadores/monitores?page_number=9
https://www.kabum.com.br/computadores/monitores?page_number=10
https://www.kabum.com.br/computadores/monitores?page_number=11
https://www.kabum.com.br/computadores/monitores?page_number=12
https://www.kabum.com.br/computadores/monitores?page_number=13
https://www.kabum.com.br/computadores/monitores?page_number=14
https://www.kabum.com.br/computadores/monitores?page_number=15
https://www.kabum.com.br/computadores/monitores?page_number=16
https://www.kabum.com.br/computadores/monitores?page_number=17
